In [ ]:
cd CIFAR-10/AlexNet

In [4]:
import numpy as np
import sys, os, random
import matplotlib.pyplot as plt
import pickle, gzip
from tqdm import tqdm,tqdm_notebook
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [5]:
unit = 10 
qp = 0.05
numEpochs = 101
seed_num = 1000
batch_size = 128

LEARNING_RATE = 1e-5 # 1e-4 learning rate for all possible weights
HYPERPARAMETER_T = qp # 0.11 hyperparameter for quantization

torch.cuda.is_available()

torch.manual_seed(seed_num)
torch.cuda.manual_seed(seed_num)
np.random.seed(seed_num)
random.seed(seed_num)
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic =True

from torchvision import datasets, transforms


In [6]:
batch_size = 128
seed_num = 81

# For reproducibility when you run the file with .py
torch.cuda.is_available()
torch.manual_seed(seed_num)
torch.cuda.manual_seed(seed_num)
np.random.seed(seed_num)
random.seed(seed_num)
torch.backends.cudnn.benchmark = True

torch.backends.cudnn.deterministic =True

In [7]:
# Normalization
train_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 
test_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

# Splitting the training and test datasets
train_data = datasets.CIFAR10(os.getcwd(), train=True,
                              download=True, transform=train_transform)
test_data = datasets.CIFAR10(os.getcwd(), train=False,
                             download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
# Split the training set indices into training and validation set indices using 80:20 ratio
np.random.seed(seed_num)
len_trainset = len(train_data)
index_list = list(range(len_trainset))
np.random.shuffle(index_list)
split_index = 40000
train_indices, valid_indices =  index_list[:split_index], index_list[split_index:]

# Creating Samplers for training and validation set using the indices
np.random.seed(seed_num)
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)

torch.manual_seed(seed_num)

train_iterator = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
val_iterator = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
test_iterator = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [9]:
# AlexNet Model

class AlexNet(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1),     # in_channels, out_channels, kernel_size, stride, padding
            nn.BatchNorm2d(32),
            nn.ReLU(inplace = True),
            nn.Conv2d(32, 64, 3, padding = 1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace = True),

            nn.Conv2d(64, 128, 3, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace = True),
            nn.Conv2d(128, 256, 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(256, 256, 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(256 * 7 * 7, 256),
            nn.ReLU(inplace = True),
            nn.Linear(256,256),
            nn.ReLU(inplace = True),
            nn.Linear(256, output_dim),
        )


    def forward(self, x):
        x = self.features(x)
        h = x.view(x.size(0), -1)
        x = self.classifier(h)
        
        return x

In [10]:
torch.manual_seed(seed_num)
unit=128

# Summary
model = AlexNet(10)
print("Model:\n",model)

Model:
 AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): ReLU(inplace=True)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.1, inplace=False)
    (1): Linear(in_features=1254

In [11]:
# Layer names
layer_name = [n for n, p in model.named_parameters()]
layer_name

['features.0.weight',
 'features.0.bias',
 'features.1.weight',
 'features.1.bias',
 'features.3.weight',
 'features.3.bias',
 'features.6.weight',
 'features.6.bias',
 'features.7.weight',
 'features.7.bias',
 'features.9.weight',
 'features.9.bias',
 'features.11.weight',
 'features.11.bias',
 'classifier.1.weight',
 'classifier.1.bias',
 'classifier.3.weight',
 'classifier.3.bias',
 'classifier.5.weight',
 'classifier.5.bias']

In [12]:
# Loading the weights of ternary model 
model = torch.load("AlexNet_cifar10_Quant_main.pt") 
model = model.cuda()
print("Loading weights done !")

# Total number of ternary weights (+w, -w)
totalParams = 0
for i in layer_name:
  print(i,(model.state_dict()[i] !=0).sum())
  totalParams +=  (model.state_dict()[i] !=0).sum()
    
print("Total Parameters:",totalParams, '\n')

Loading weights done !
features.0.weight tensor(528, device='cuda:0')
features.0.bias tensor(29, device='cuda:0')
features.1.weight tensor(32, device='cuda:0')
features.1.bias tensor(13, device='cuda:0')
features.3.weight tensor(915, device='cuda:0')
features.3.bias tensor(37, device='cuda:0')
features.6.weight tensor(955, device='cuda:0')
features.6.bias tensor(115, device='cuda:0')
features.7.weight tensor(64, device='cuda:0')
features.7.bias tensor(48, device='cuda:0')
features.9.weight tensor(2010, device='cuda:0')
features.9.bias tensor(52, device='cuda:0')
features.11.weight tensor(2931, device='cuda:0')
features.11.bias tensor(143, device='cuda:0')
classifier.1.weight tensor(12705, device='cuda:0')
classifier.1.bias tensor(96, device='cuda:0')
classifier.3.weight tensor(1384, device='cuda:0')
classifier.3.bias tensor(142, device='cuda:0')
classifier.5.weight tensor(1394, device='cuda:0')
classifier.5.bias tensor(8, device='cuda:0')
Total Parameters: tensor(23601, device='cuda:0'

In [13]:
# Model's performance on test set

correct_count, all_count = 0, 0
model.eval()
for images,labels in test_iterator:
      for image,label in zip(images,labels):

        if torch.cuda.is_available():
            img = image.cuda()
            lab = label.cuda()
            img = img[None,].type('torch.cuda.FloatTensor')

        with torch.no_grad():
            output_ = model(img) 

        pred_label = output_.argmax()

        if(pred_label.item()==lab.item()):
          correct_count += 1
        all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Test Accuracy =", (correct_count/(all_count)))


Number Of Images Tested = 10000

Model Test Accuracy = 0.5396


In [14]:
# Duplicate architecture of AlexNet Model

class AlexNet1(nn.Module):
    def __init__(self, output_dim, dn_info, dn_info1):
        super().__init__()
        
        self.dn_info = dn_info       # Dead Neuron info
        self.dn_info1 = dn_info1       # Dead Neuron info
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1), #in_channels, out_channels, kernel_size, stride, padding
            nn.BatchNorm2d(32),
            nn.ReLU(inplace = True),
            nn.Conv2d(32, 64, 3, padding = 1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace = True),
            nn.Conv2d(64, 128, 3, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace = True),
            nn.Conv2d(128, 256, 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(256, 256, 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size=2, stride=2),)
        
        
        self.relu = nn.ReLU(inplace = True)
        
        self.dp = nn.Dropout(0.1)
        
        self.fc1 = nn.Linear(256 * 7 * 7, 256, bias= True)
        
        # Fully connected 2
        self.fc3 = nn.Linear(256, 256,bias= True) 

        # Fully connected 3
        self.fc5 = nn.Linear(256,output_dim, bias= True) 
        

    def forward(self, x):
        x = self.features(x)
        h = x.view(x.size(0), -1)
        h = self.dp(h)
        x = self.fc1(h)
        
        x = self.relu(x)
        
        ###################################
        
        # Storing dead neurons indices
        idx = torch.where(x.cpu() == 0.)[1]
        
        for i in idx:
            self.dn_info[str(i.item())] += 1
            
        #####################################
        
        x = self.fc3(x)
        x = self.relu(x)
        
        ####################################
        # Storing dead neurons indices
        idx1 = torch.where(x.cpu() == 0.)[1]
        
        for j in idx1:
            self.dn_info1[str(j.item())] += 1
            
        #####################################
        
        x =  self.fc5(x)
        
        return x


In [15]:
keys = list(AlexNet1(10, {}, {}).state_dict().keys())
print(keys)

['features.0.weight', 'features.0.bias', 'features.1.weight', 'features.1.bias', 'features.1.running_mean', 'features.1.running_var', 'features.1.num_batches_tracked', 'features.3.weight', 'features.3.bias', 'features.6.weight', 'features.6.bias', 'features.7.weight', 'features.7.bias', 'features.7.running_mean', 'features.7.running_var', 'features.7.num_batches_tracked', 'features.9.weight', 'features.9.bias', 'features.11.weight', 'features.11.bias', 'fc1.weight', 'fc1.bias', 'fc3.weight', 'fc3.bias', 'fc5.weight', 'fc5.bias']


In [16]:
from collections import OrderedDict

weights_q = OrderedDict()

for n in model.state_dict().keys():
    if n in keys:
        weights_q[n] = model.state_dict()[n].clone()
        weights_q[n].requires_grad=False
    
    elif n.startswith("classifier"):
        weights_q['fc' + n[11:]] = model.state_dict()[n].clone()
        weights_q['fc' + n[11:]].requires_grad=False

state_dict = weights_q

In [17]:
print(state_dict.keys())

odict_keys(['features.0.bias', 'features.0.weight', 'features.1.bias', 'features.1.weight', 'features.1.running_mean', 'features.1.running_var', 'features.1.num_batches_tracked', 'features.3.bias', 'features.3.weight', 'features.6.bias', 'features.6.weight', 'features.7.bias', 'features.7.weight', 'features.7.running_mean', 'features.7.running_var', 'features.7.num_batches_tracked', 'features.9.bias', 'features.9.weight', 'features.11.bias', 'features.11.weight', 'fc1.bias', 'fc1.weight', 'fc3.bias', 'fc3.weight', 'fc5.bias', 'fc5.weight'])


In [18]:
def getDeadN_info(dn_info, dn_info1, unit, state_dict, ds, nameOftheSet):

  for i in range(unit):
      dn_info[str(i)] = 0
      dn_info1[str(i)] = 0

      
  model1 = AlexNet1(10, dn_info, dn_info1)
  model1 = model1.cuda()

  model1.load_state_dict(state_dict)

  correct_count, all_count = 0, 0
  model1.eval()

  for ds_e in ds:
    for images,labels in ds_e:
          for image,label in zip(images,labels):

            if torch.cuda.is_available():
                img = image.cuda()
                lab = label.cuda()
                img = img[None,].type('torch.cuda.FloatTensor')

            with torch.no_grad():
                output_ = model1(img) 

            pred_label = output_.argmax()

            if(pred_label.item()==lab.item()):
              correct_count += 1
            all_count += 1

  print("Number Of Images =", all_count)
  print(f"Model {nameOftheSet} Accuracy =", (correct_count/(all_count)))

In [19]:
dn_info_train = {}
dn_info_train1 = {}

getDeadN_info(dn_info= dn_info_train, dn_info1= dn_info_train1, unit = 256, state_dict = state_dict, ds= [train_iterator, val_iterator, test_iterator], nameOftheSet = "Training")

Number Of Images = 60000
Model Training Accuracy = 0.5504833333333333


In [20]:
dn_info_test = {}
dn_info_test1 = {}

getDeadN_info(dn_info= dn_info_test, dn_info1= dn_info_test1, unit = 256, state_dict = state_dict, ds= [test_iterator], nameOftheSet = "Training")

Number Of Images = 10000
Model Training Accuracy = 0.5396


In [21]:
max_dn_train = max(dn_info_train.values())
max_dn_train1 = max(dn_info_train1.values())

max_dn_train, max_dn_train1

(60000, 60000)

In [22]:
dead_n_idx = [] 

for i, j in dn_info_train.items():
  if j == max_dn_train:
    dead_n_idx.append(i)

dead_n_idx1 = [] 

for i, j in dn_info_train1.items():
  if j == max_dn_train1:
    dead_n_idx1.append(i)

In [23]:
print("Penultimate Layer :",len(dead_n_idx)) # Number of neurons that are dead
print("Output Layer :",len(dead_n_idx1)) # Number of neurons that are dead

Penultimate Layer : 155
Output Layer : 86


In [24]:
print(dead_n_idx) # Indices of neuron that are dead

['1', '3', '4', '5', '7', '8', '9', '13', '15', '16', '17', '18', '20', '21', '22', '23', '24', '26', '27', '32', '33', '34', '35', '36', '40', '42', '44', '45', '46', '47', '50', '51', '54', '55', '57', '58', '59', '60', '61', '62', '63', '64', '67', '68', '69', '70', '73', '75', '76', '77', '81', '85', '86', '88', '89', '90', '91', '94', '95', '96', '97', '98', '102', '104', '106', '107', '108', '109', '110', '112', '114', '116', '117', '118', '121', '122', '124', '125', '126', '128', '131', '132', '134', '135', '136', '139', '140', '142', '145', '147', '149', '151', '152', '153', '154', '157', '160', '161', '162', '163', '164', '165', '166', '168', '169', '172', '178', '179', '180', '183', '185', '186', '188', '189', '191', '194', '195', '198', '199', '200', '203', '204', '206', '208', '209', '211', '213', '214', '215', '216', '217', '219', '220', '221', '223', '225', '227', '228', '229', '230', '233', '234', '235', '239', '240', '241', '244', '246', '247', '248', '249', '250', '251

In [25]:
print(dead_n_idx1) # Indices of neuron that are dead

['9', '11', '12', '14', '15', '16', '18', '21', '24', '25', '26', '27', '31', '34', '35', '36', '38', '40', '45', '50', '52', '54', '57', '59', '61', '65', '69', '72', '74', '75', '79', '80', '84', '87', '90', '91', '92', '93', '94', '97', '98', '99', '107', '109', '110', '113', '115', '117', '121', '122', '127', '132', '138', '139', '141', '144', '147', '154', '158', '163', '170', '174', '175', '180', '181', '183', '187', '188', '189', '191', '193', '202', '209', '212', '214', '225', '226', '230', '235', '238', '242', '243', '245', '247', '250', '253']


In [26]:
state_dict1 = state_dict

In [27]:
# For each layer, model's ternary weights

layer_distinct_weights = {}

for i in keys:
  if ("mean" in i) | ("var" in i) | ('batches' in i):
    continue
  else:
    imd = torch.unique(state_dict[i])
    print(i+ ' hidden layer dimension', state_dict[i].shape)
    print("Unique values of weight in "+ i+ " the hidden layer : ", imd)
    layer_distinct_weights[i] = imd.cpu().numpy().tolist()
    print()

features.0.weight hidden layer dimension torch.Size([32, 3, 3, 3])
Unique values of weight in features.0.weight the hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

features.0.bias hidden layer dimension torch.Size([32])
Unique values of weight in features.0.bias the hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

features.1.weight hidden layer dimension torch.Size([32])
Unique values of weight in features.1.weight the hidden layer :  tensor([1.], device='cuda:0')

features.1.bias hidden layer dimension torch.Size([32])
Unique values of weight in features.1.bias the hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

features.3.weight hidden layer dimension torch.Size([64, 32, 3, 3])
Unique values of weight in features.3.weight the hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

features.3.bias hidden layer dimension torch.Size([64])
Unique values of weight in features.3.bias the hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

features.6.wei

In [28]:
for i in keys:
  print(i, state_dict[i].shape)

features.0.weight torch.Size([32, 3, 3, 3])
features.0.bias torch.Size([32])
features.1.weight torch.Size([32])
features.1.bias torch.Size([32])
features.1.running_mean torch.Size([32])
features.1.running_var torch.Size([32])
features.1.num_batches_tracked torch.Size([])
features.3.weight torch.Size([64, 32, 3, 3])
features.3.bias torch.Size([64])
features.6.weight torch.Size([128, 64, 3, 3])
features.6.bias torch.Size([128])
features.7.weight torch.Size([128])
features.7.bias torch.Size([128])
features.7.running_mean torch.Size([128])
features.7.running_var torch.Size([128])
features.7.num_batches_tracked torch.Size([])
features.9.weight torch.Size([256, 128, 3, 3])
features.9.bias torch.Size([256])
features.11.weight torch.Size([256, 256, 3, 3])
features.11.bias torch.Size([256])
fc1.weight torch.Size([256, 12544])
fc1.bias torch.Size([256])
fc3.weight torch.Size([256, 256])
fc3.bias torch.Size([256])
fc5.weight torch.Size([10, 256])
fc5.bias torch.Size([10])


In [29]:
total_trans = 0
layers_name = ['fc3.weight', 'fc5.weight']

n_idx = [dead_n_idx, dead_n_idx1]

for ix,l in enumerate(layers_name):
  print(l)
  z = state_dict1[l]

  if len(layer_distinct_weights[l]) > 2 :
    w_neg, w_0, w_pos =  layer_distinct_weights[l]
    print(w_neg, w_0, w_pos)
  else:
    w_neg, w_pos = layer_distinct_weights[l]
    print(w_neg, w_pos)

  for idx in n_idx[ix]:
      if 'bias' in l:
        imd = z[eval(idx)]
      else:
        imd = z[:,eval(idx)]

      trans = torch.where(imd == w_neg)[0].nelement() + torch.where(imd == w_pos)[0].nelement()
      total_trans += trans *2
    
print("Total dead transitions for the layer :", total_trans)

fc3.weight
-1.0 0.0 1.0
fc5.weight
-1.0 0.0 1.0
Total dead transitions for the layer : 958


In [ ]:
# Fault coverage that we have obtained from main file :  29425/ 47106

In [30]:
# Adding more 958 transitions to the numerator will make the net fault coverage
(29425 + 958)/47106

0.6449921453742623